In [6]:
import numpy as np
import os
import pathlib
from pydrake.all import (StartMeshcat, MeshcatVisualizer, DiagramBuilder, Parser,
                        Simulator, LeafSystem, RigidTransform, RotationMatrix,
                        UniformlyRandomRotationMatrix, RandomGenerator, Integrator, 
                        AddMultibodyPlantSceneGraph)

from manipulation import ConfigureParser
from manipulation.scenarios import MakeManipulationStation

from subsystems import (ClosedLoopPseudoInverseController, Vision, GraspSelector, GarbageType, Planner)

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [3]:
rs = np.random.RandomState()  # this is for python
generator = RandomGenerator(rs.randint(1000)) # this is for c++

path = os.getcwd()
MODEL_YAML = pathlib.Path(path + "/models/recycling.dmd.yaml").as_uri()
MODEL_PATH = 'recycling_maskrcnn_model.pt'
    
# Contains iiwa, bins, table, floor
INTERNAL_YAML = pathlib.Path(path + "/models/internal_model.dmd.yaml").as_uri()
def make_internal_model():
    station = MakeManipulationStation(
        filename=INTERNAL_YAML,
        package_xmls=["./package.xml"])
    return station

# Contains table & trash
TRASH_YAML = pathlib.Path(path + "/models/trash_model.dmd.yaml").as_uri()
def make_trash_model():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    parser.package_map().AddPackageXml('./package.xml')
    ConfigureParser(parser)
    parser.AddModelsFromUrl(TRASH_YAML)
    plant.Finalize()
    return builder.Build()

q0 = [-1.57, -0.1, 0, -1.4, 0, 1.6, 0]
X_WHome = RigidTransform(
        RotationMatrix([
            [1, 0, 0],
            [0, 0, 1],
            [0, -1, 0]
        ]),
        [0, -0.5, 0.65])
X_WRecycle = RigidTransform(
        RotationMatrix([
            [0, 0, -1],
            [1, 0, 0],
            [0, -1, 0]
        ]),
        [0.4, -0.25, 0.65])
X_WTrash = RigidTransform(
        RotationMatrix([
            [0, 0, -1],
            [1, 0, 0],
            [0, -1, 0]
        ]),
        [0.4, -.1, 0.65])
X_WOrganic = RigidTransform(
        RotationMatrix([
            [0, 0, -1],
            [1, 0, 0],
            [0, -1, 0]
        ]),
        [0.4, 0.05, 0.65])

ITEM_NAMES = ["empty", "bottle", "Banana", "coffee"]
GARBAGE_MAP = {"bottle": GarbageType.RECYCLE, 
               "Banana": GarbageType.ORGANIC,
               "coffee": GarbageType.TRASH}
Xs = {'Home': X_WHome, GarbageType.TRASH: X_WTrash, 
      GarbageType.RECYCLE: X_WRecycle, GarbageType.ORGANIC: X_WOrganic}

## Define GreenBot system

In [7]:
class GreenBot(LeafSystem):
    def __init__(self):
        LeafSystem.__init__(self)
            
        # Setup diagram builder components
        builder = DiagramBuilder()
        self.station = MakeManipulationStation(
            filename=MODEL_YAML,
            package_xmls=["./package.xml"])
        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat)  
        ## Uncomment to add collision geometry visualizer
        # self.collision = MeshcatVisualizer.AddToBuilder(
        #     builder, self.station.GetOutputPort("query_object"), meshcat,
        #     MeshcatVisualizerParams(role=Role.kProximity, prefix="collision"))

        # Add vision system
        vision = builder.AddNamedSystem("vision", 
            Vision(self.station,
                   camera_body_indices=[
                      self.plant.GetBodyIndices(
                          self.plant.GetModelInstanceByName("camera0"))[0],
                      self.plant.GetBodyIndices(
                          self.plant.GetModelInstanceByName("camera1"))[0]
                  ],
                  model_path=MODEL_PATH,
                  item_names=ITEM_NAMES))
        builder.Connect(self.station.GetOutputPort("camera0_depth_image"),
                        vision.get_input_port(0))
        builder.Connect(self.station.GetOutputPort("camera1_depth_image"),
                        vision.get_input_port(1))
        builder.Connect(self.station.GetOutputPort("camera0_rgb_image"),
                        vision.get_input_port(2))
        builder.Connect(self.station.GetOutputPort("camera1_rgb_image"),
                        vision.get_input_port(3))
        builder.Connect(self.station.GetOutputPort("body_poses"),
                        vision.GetInputPort("body_poses"))
        
        # Add grasp selector
        internal_model = make_internal_model()
        grasp_selector = builder.AddNamedSystem("grasp_selector", 
            GraspSelector(internal_model, Xs['Home']))
        builder.Connect(vision.GetOutputPort("point_cloud_W"),
            grasp_selector.GetInputPort("point_cloud_W"))
        
        # Planner
        planner = builder.AddNamedSystem("planner", Planner(self.plant, Xs, GARBAGE_MAP))
        builder.Connect(self.station.GetOutputPort("body_poses"),
                    planner.GetInputPort("body_poses"))
        builder.Connect(grasp_selector.GetOutputPort("grasp_selection"),
                        planner.GetInputPort("grasp_selection"))
        builder.Connect(self.station.GetOutputPort("wsg_state_measured"),
                        planner.GetInputPort("wsg_state"))
        builder.Connect(self.station.GetOutputPort("iiwa_position_measured"),
                        planner.GetInputPort("iiwa_position"))

        # Connect trajectories to pseudo-inverse controller
        self.controller = builder.AddSystem(ClosedLoopPseudoInverseController(self.plant))
        self.controller.set_name("ClosedLoopPseudoInverseController")
        builder.Connect(planner.GetOutputPort("T_WG"), self.controller.GetInputPort("T_WG"))
        
        # Integrate controller velocity commands to get joint angles
        self.integrator = builder.AddSystem(Integrator(7))
        self.integrator.set_name("integrator")
        builder.Connect(self.controller.get_output_port(),
                        self.integrator.get_input_port())
        builder.Connect(self.integrator.get_output_port(),
                        self.station.GetInputPort("iiwa_position"))
        builder.Connect(self.station.GetOutputPort("iiwa_position_measured"),
                        self.controller.GetInputPort("iiwa_position"))
        
        # Gripper control
        builder.Connect(planner.GetOutputPort("wsg_position"),
                    self.station.GetInputPort("wsg_position"))
        
        # Finalize
        self.diagram = builder.Build()
        self.context = self.diagram.CreateDefaultContext()
        
        # Set current position
        self.integrator.set_integral_value(
            self.integrator.GetMyMutableContextFromRoot(self.context), q0)
            
        # Randomize poses of trash
        self._trash_model = make_trash_model()
        self.RandomizeTrash()
        
    def RandomizeTrash(self, num_iterations=40):
        """
        Continually randomize the poses of the trash until a collision-free configuration has been found.
        Stops after num_iterations.
        """
        
        trash_context = self._trash_model.CreateDefaultContext()
        trash_plant = self._trash_model.GetSubsystemByName("plant")
        trash_plant_context = trash_plant.GetMyMutableContextFromRoot(trash_context)
        trash_scene_graph = self._trash_model.GetSubsystemByName("scene_graph")
        trash_scene_graph_context = trash_scene_graph.GetMyMutableContextFromRoot(trash_context)
        query_object = trash_scene_graph.get_query_output_port().Eval(trash_scene_graph_context)
        
        iterate = True
        counter = 0
        body_tfs = {}
        while iterate:
            for body_index in trash_plant.GetFloatingBaseBodies():
                body = trash_plant.get_body(body_index)
                if body.name() in ITEM_NAMES:
                    tf = RigidTransform(
                            UniformlyRandomRotationMatrix(generator),
                            [0.75*np.random.rand() - 0.375, 0.16*np.random.rand() - 0.08 -.6, .44])
                    trash_plant.SetFreeBodyPose(trash_plant_context, body, tf)
                    body_tfs[body.name()] = tf
                    
            iterate = query_object.HasCollisions()
            counter += 1
            if counter > num_iterations:
                print("Large amount of consecutive failures, stopping...")
                break
        
        plant_context = self.plant.GetMyMutableContextFromRoot(self.context)
        if not query_object.HasCollisions():
            print(f"Objects randomized successfully after {counter} tries")
            for body_index in self.plant.GetFloatingBaseBodies():
                body = self.plant.get_body(body_index)
                if body.name() in ITEM_NAMES:
                    self.plant.SetFreeBodyPose(plant_context, body, body_tfs[body.name()])
                    
    def GetNumPlaces(self): 
        
        plant_context = self.plant.GetMyMutableContextFromRoot(self.context)
        num = 0
        for body_index in self.plant.GetFloatingBaseBodies():
            body = self.plant.get_body(body_index)
            if body.name() in ITEM_NAMES:
                X_WB = self.plant.GetFreeBodyPose(plant_context, body)
                garbage_type = GARBAGE_MAP[body.name()]
                X_WD = Xs[garbage_type]
                p_WD = X_WD.translation() * np.array([1,1,0])
                if np.all(np.isclose(X_WB.translation(), p_WD, atol=0.125)):
                    num += 1
        
        return num  
        
    def Simulate(self, t, record=False):
        
        # Simulator
        simulator = Simulator(self.diagram, self.context)
        simulator.set_target_realtime_rate(1.0)
        if record:
            self.visualizer.StartRecording()
            simulator.AdvanceTo(t)
            self.visualizer.PublishRecording()
        else:
            simulator.AdvanceTo(t)
                

In [12]:
# Run one trial
meshcat.Delete()
bot = GreenBot()
bot.Simulate(45)
print(f"Number of successful placements: {bot.GetNumPlaces()}")

Objects randomized successfully after 1 tries
Selecting grasp
Decided to pick up  bottle
Skipping:  Normal has magnitude: nan
Selecting grasp
Decided to pick up  coffee
Selecting grasp
Decided to pick up  Banana
Number of successful placements: 3


## Run tests

Let's run 100 tests and see how GreenBot performs. We'll keep track of how many objects GreenBot correctly sorts each trial and log the results in a csv file.

In [4]:
from csv import writer

num_tests = 100
for i in range(num_tests):
    status = 'N/A'
    num_success = 'N/A'
    try:
        meshcat.Delete()
        green = GreenBot()
        green.Simulate(60)
        status = 'Completed'
        num_success = green.GetNumPlaces()
    except Exception as e:
        print(e)
        if str(e)[:12] == 'RuntimeError':
            status = 'Simulation Error'
            num_success = green.GetNumPlaces()
        else:
            status = 'Other Error'
            num_success = green.GetNumPlaces()
            
    results = [i, status, num_success]
    print(results)
            
    with open('test_results.csv', 'a') as f:  
        w = writer(f)
        w.writerow(results)
        f.close()

[Errno 2] No such file or directory: 'recycling_maskrcnn_model.pt'


NameError: name 'green' is not defined